In [ ]:
import requests
import json
import datetime as dt
import time
import random 

all_tweets = json.loads(open("../data_files/tweets_with_ships.json", "r", encoding="utf-8").read())
locations = json.loads(open("../data_files/location_coordinates.json", "r", encoding="utf-8").read())
ships = json.loads(open("../data_files/ships3.json", "r", encoding="utf-8").read())
API_BASE_URL = "https://api.meteomatics.com"
username = ""
password = ""
last_date = dt.datetime.fromisoformat("2018-07-29")
location_names = [location_name for location_name in locations.keys()]




In [ ]:
tweet = tweets[0]
d = dt.datetime.fromisoformat(tweet["created_at"])
new = date - dt.timedelta(hours=6)
print(d.date())

In [ ]:
tweet = tweets[0]
d = dt.datetime.fromisoformat(tweet["created_at"]) - dt.timedelta(hours=6)
hour = round(date.hour / 6) * 6 % 24
print(hour)
print(sum([15.7, 15.5, 15.3, 15.0, 14.2, 14.3, 14.4])/len([15.7, 15.5, 15.3, 15.0, 14.2, 14.3, 14.4]))

In [ ]:
new_data = []
last_date = dt.datetime.fromisoformat("2018-07-29")
date_checking = dt.datetime.fromisoformat("2019-07-31")
locations_checked_for_day = []
start = time.time()

for index, tweet in enumerate(tweets):
    if index % 20 == 0:
        print(time.time() - start)
        print(index)
    
    date = dt.datetime.fromisoformat(tweet["created_at"])    
    if date < last_date:
        break
    
    if not tweet["ship"]:
        continue
    
    if date.date() < date_checking.date():
        for i in range(3):
            location = random.choice(location_names)
            while location in locations_checked_for_day:
                location = random.choice(location_names)
            
            rounded_lat = round(locations[location]["lat"], 3)
            rounded_lng = round(locations[location]["lng"], 3)
            url = f"{API_BASE_URL}/{date_checking.isoformat()}ZPT6H:PT1H/t_2m:C,precip_1h:mm,wind_speed_10m:ms,msl_pressure:hPa,relative_humidity_2m:p/{rounded_lat},{rounded_lng}/json"

            response = requests.get(url, timeout=300, auth=(username, password))
            res = json.loads(response.text)
            new_dict = {
                "date": str(date_checking.month) + "-" + str(date_checking.day),        
                "lat": rounded_lat,
                "lng": rounded_lng,
                "class": 0
            }
            for data in res["data"]:
                values = [dates["value"] for dates in data["coordinates"][0]["dates"]]
                new_dict[data["parameter"]] = values
            new_data.append(new_dict) 
        date_checking = date
        locations_checked_for_day = []
    
    
    rounded_hour = (round(date.hour / 6) * 6) % 24 # Round to closest 6 hours 
    rounded_date = date.replace(hour=rounded_hour, minute=0, second=0)
    try:
        
        rounded_lat = round(locations[tweet["location"]]["lat"], 3) if tweet["location"] != "" else round(locations[ships[tweet["ship"]]["Havn"]]["lat"], 3)
        rounded_lng = round(locations[tweet["location"]]["lng"], 3) if tweet["location"] != "" else round(locations[ships[tweet["ship"]]["Havn"]]["lng"], 3)
    except KeyError as e:
        print("Key error", e)
        continue
        
    new_dict = {
        "date": str(rounded_date.month) + ":" + str(rounded_date.day),        
        "lat": rounded_lat,
        "lng": rounded_lng,
        "ship": tweet["ship"],
        "class": 1
    }
    
    url_date = rounded_dat - dt.timedelta(hours=6)
    url_date = url_date.isoformat("T") + "Z"
    url = f"{API_BASE_URL}/{url_date}PT6H:PT1H/t_2m:C,precip_1h:mm,wind_speed_10m:ms,msl_pressure:hPa,relative_humidity_2m:p/{rounded_lat},{rounded_lng}/json"
    
    response = requests.get(url, timeout=300, auth=(username, password))
    res = json.loads(response.text)
    
    for data in res["data"]:
        values = [dates["value"] for dates in data["coordinates"][0]["dates"]]
        new_dict[data["parameter"]] = values
    
    new_data.append(new_dict)
    
    locations_checked_for_day.append(tweet["location"])

print(time.time() - start)


In [ ]:
with open("../data_files/tweets_with_weather_data.json", "w", encoding="utf-8") as file:
    file.write(json.dumps(new_data))